In [331]:
import os
import pandas as pd
import numpy as np

#_______________________________________________________________
# Function to fix some bugs
def process_dataframe(df):
    # Check if values from first column to column 1289 in the last row are NaN
    if df.iloc[-1, :1289].isna().all():
        # Assign the values contained in columns 1290 and 1291 of the last row to variables
        col_1290_value = df.iloc[-1, 1289]
        col_1291_value = df.iloc[-1, 1290]
        
        # Delete the last row
        df = df.iloc[:-1, :]
        
        # Assign the values to the new last row
        df.iloc[-1, 1289] = col_1290_value
        df.iloc[-1, 1290] = col_1291_value
    
    return df
#__________________________________________________________________
# Directory containing the txt files
directory = r'C:\Users\e2023898\Downloads\test'

# List to store dataframes
dataframes = []

# Loop through all files in the directory
for i, filename in enumerate(os.listdir(directory)):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        try:
            # First method to read the file
            df = pd.read_csv(file_path, delimiter="\t", header=None)
            df = df.drop(17).reset_index(drop=True)
        except Exception as e:
            # Second method to read the file if the first fails
            total_columns = 321
            df1 = pd.read_csv(file_path, header=None, engine='python', encoding='latin', sep='\t', nrows=17)
            df2 = pd.read_csv(file_path, header=None, engine='python', encoding='latin', sep='\t', skiprows=18)
            df = pd.concat([df1, df2], axis=0).reset_index(drop=True)
        
        # Add 'File name' and 'Last modification' columns at row 20
        df.loc[20, 'File name'] = filename
        df.loc[20, 'Last modification'] = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%d/%m/%Y %H:%M:%S')
        
        # Append the dataframe to the list
        dataframes.append(df)
        
        # Assign the dataframe to a variable name sequentially
        globals()[f'df_{i+1}'] = df

        # Print the names of the dataframes
for i, df in enumerate(dataframes):
    print(f'df_{i+1}')

df_1
df_2
df_3
df_4
df_5
df_6


In [328]:
# Function to append columns to riassunto dataframes
def append_columns_to_riassunto(riassunto, columns, values):
    for col, val in zip(columns, values):
        riassunto[col] = val
    return riassunto

riassunto_dataframes = []


# Process each dataframe and create riassunto dataframes
for i, df in enumerate(dataframes):
    # Split the dataframe
    test_id = df.iloc[:17]
    test_id = test_id.loc[:, test_id.notna().any(axis=0)]
    index_row = df.iloc[17]
    remaining_df = df.iloc[17:]
    test_data = test_id.transpose()
    test_data.columns = test_data.iloc[0]
    test_data = test_data[1:]
    test_data_columns = test_data.columns.tolist()
    test_data_values = test_data.iloc[0].tolist()

    # Initialize variables
    datasets = {}
    current_dataset = []
    dataset_counter = 1
    note_counter = 0

    # Process the remaining rows
    for index, row in remaining_df.iterrows():
        if row[0] == "Note":
            note_counter += 1
            if note_counter > 1:
                datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)
                dataset_counter += 1
                current_dataset = []
        current_dataset.append(row)

    # Append the last dataset if exists
    if current_dataset:
        datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)

    # Create a dataframe for each dataset and show the names of the dataframes created
    dataframe_names = []
    for key, dataset in datasets.items():
        globals()[key] = dataset
        dataframe_names.append(key)

    # Apply the operations to each dataset
    for key, dataset in datasets.items():
        # Convert the second column to datetime, coercing errors
        dataset.iloc[3:, 1] = pd.to_datetime(dataset.iloc[3:, 1], format='%d/%m/%Y %H:%M:%S', errors='coerce')
        
        # Replace all commas with periods in columns 2 to the end
        dataset.iloc[:, 1:] = dataset.iloc[:, 1:].replace(',', '.', regex=True)
        
        # Convert subsequent columns to float, coercing errors
        for col in dataset.columns[2:]:
            dataset.iloc[2:, dataset.columns.get_loc(col)] = pd.to_numeric(dataset.iloc[2:, dataset.columns.get_loc(col)], errors='coerce')
        
        # Update the dataset in the dictionary
        datasets[key] = dataset

    # Create the dataframe for storing general data
    riassunto = pd.DataFrame(columns=index_row)
    new_columns = []
    riassunto = pd.concat([riassunto, pd.DataFrame(columns=new_columns)], axis=1)

 # Calculate and assign time delta values for the second column of riassunto
    for key, dataset in datasets.items():
        # Ensure the values are Timestamps
        start_time = pd.to_datetime(dataset.iloc[3, 1], errors='coerce')
        end_time = pd.to_datetime(dataset.iloc[-1, 1], errors='coerce')
        time_delta = end_time - start_time
        row_index = dataframe_names.index(key)
        riassunto.at[row_index, riassunto.columns[1]] = time_delta

    # Assign values from datasets to riassunto sequentially in column 1
    row_counter = 0
    for key, dataset in datasets.items():
        value_to_assign = dataset.iloc[1, 0]  # Value from column 1 row 2
        riassunto.at[row_counter, 'Note'] = value_to_assign
        row_counter += 1

    # Iterate over each dataset
    for key, dataset in datasets.items():
        # Calculate the average values for each column starting from the third column
        averages = dataset.iloc[2:, 2:].mean()
        
        # Assign the average values to the corresponding row in riassunto
        row_index = dataframe_names.index(key)
        for col_index, avg_value in enumerate(averages, start=2):
            riassunto.iloc[row_index, col_index] = avg_value

    # Create new columns for minimum and maximum values
    avg_columns = []
    min_columns = []
    max_columns = []

    for col in riassunto.columns[1:]:
        avg_columns.append(f"{col}_std")
        min_columns.append(f"{col}_min")
        max_columns.append(f"{col}_max")
        

    # Append new columns to riassunto dataframe
    riassunto = pd.concat([riassunto, pd.DataFrame(columns=avg_columns + min_columns + max_columns)], axis=1)

    # Iterate over each dataset
    for key, dataset in datasets.items():
        # Calculate the minimum and maximum values for each column starting from the third column
        deviations = dataset.iloc[2:, 2:].std()
        min_values = dataset.iloc[2:, 2:].min()
        max_values = dataset.iloc[2:, 2:].max()
        
        # Assign the minimum values to the corresponding row in riassunto
        row_index = dataframe_names.index(key)
        for col_index, deviations in enumerate(deviations, start=2):
            riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = deviations
        
        for col_index, min_value in enumerate(min_values, start=2):
            riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = min_value
        
        # Assign the maximum values to the corresponding row in riassunto
        for col_index, max_value in enumerate(max_values, start=2):
            riassunto.iloc[row_index, 2*len(riassunto.columns)//3 + col_index] = max_value


    # Assign 'File name' and 'Last modification' values from row 20 of each dataframe
    riassunto['File name'] = df.at[20,'File name']
    riassunto['Last modification'] = df.at[20,'Last modification']
    #riassunto.at[20, 'File name'] = df.at[20, 'File name']
    #riassunto.at[20, 'Last modification'] = df.at[20, 'Last modification']
    updated_riassunto = append_columns_to_riassunto(riassunto, test_data_columns, test_data_values) 


    # Assign the riassunto dataframe to a variable name sequentially
    globals()[f'riassunto_{i+1}'] = updated_riassunto
    riassunto_dataframes.append(updated_riassunto)

for i in range(1, len(globals())):
    var_name = f'riassunto_{i}'
    if var_name in globals():
        globals()[var_name] = process_dataframe(globals()[var_name])

# Print the processed riassunto dataframes for verification
for i in range(1, len(globals())):
    var_name = f'riassunto_{i}'

header = riassunto_1.columns.tolist()
# Loop through each DataFrame and replace column names with values from 'header'
for i, df in enumerate(riassunto_dataframes):
    df.columns = header

for i in range(len(riassunto_dataframes)):
    riassunto_dataframes[i]=riassunto_dataframes[i].loc[~riassunto_dataframes[i].index.to_series().duplicated(keep='first')]

final_dataframe = pd.concat(riassunto_dataframes, ignore_index=True)

# Sort the dataframe by 'DATA/ORA COLLAUDO'
final_dataframe.sort_values(by='Last modification', inplace=True)

# Create a new column 'numero.test' with sequential values based on the sorted dates
final_dataframe['numero.test'] = final_dataframe.groupby('File name').ngroup().apply(lambda x: f'P{x+1:06d}')

# Ensure that rows with the same 'File Name' have the same 'numero.test'
final_dataframe['numero.test'] = final_dataframe.groupby('File name')['numero.test'].transform('first')


c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:53: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:53: RuntimeWarning: invalid value encountered in reduce
  return umr_su

In [ ]:
import os
import pandas as pd
from datetime import datetime

# Function to fix some bugs
def process_dataframe(df):
    # Check if values from first column to column 1289 in the last row are NaN
    if df.iloc[-1, :1289].isna().all():
        # Assign the values contained in columns 1290 and 1291 of the last row to variables
        col_1290_value = df.iloc[-1, 1289]
        col_1291_value = df.iloc[-1, 1290]
        
        # Delete the last row
        df = df.iloc[:-1, :]
        
        # Assign the values to the new last row
        df.iloc[-1, 1289] = col_1290_value
        df.iloc[-1, 1290] = col_1291_value
    
    return df

# Directory containing the txt files
directory = r'V:\TEST CHILLER\TXT'

# List to store dataframes and skipped files
dataframes = []
skipped_files = []

# Loop through all files in the directory
for i, filename in enumerate(os.listdir(directory)):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        try:
            # First method to read the file
            df = pd.read_csv(file_path, delimiter="\t", header=None)
            df = df.drop(17).reset_index(drop=True)
        except Exception as e:
            try:
                # Second method to read the file if the first fails
                total_columns = 321
                df1 = pd.read_csv(file_path, header=None, engine='python', encoding='latin', sep='\t', nrows=17)
                df2 = pd.read_csv(file_path, header=None, engine='python', encoding='latin', sep='\t', skiprows=18)
                df = pd.concat([df1, df2], axis=0).reset_index(drop=True)
            except Exception as e:
                # If both methods fail, skip the file and add to skipped_files list
                skipped_files.append(filename)
                continue
        
        # Add 'File name' and 'Last modification' columns at row 20
        df.loc[20, 'File name'] = filename
        df.loc[20, 'Last modification'] = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%d/%m/%Y %H:%M:%S')
        
        # Append the dataframe to the list
        dataframes.append(df)
        
        # Assign the dataframe to a variable name sequentially
        globals()[f'df_{i+1}'] = df




df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [ ]:
# Function to append columns to riassunto dataframes
def append_columns_to_riassunto(riassunto, columns, values):
    for col, val in zip(columns, values):
        riassunto[col] = val
    return riassunto

riassunto_dataframes = []


# Process each dataframe and create riassunto dataframes
for i, df in enumerate(dataframes):
    # Split the dataframe
    test_id = df.iloc[:17]
    test_id = test_id.loc[:, test_id.notna().any(axis=0)]
    index_row = df.iloc[17]
    remaining_df = df.iloc[17:]
    test_data = test_id.transpose()
    test_data.columns = test_data.iloc[0]
    test_data = test_data[1:]
    test_data_columns = test_data.columns.tolist()
    test_data_values = test_data.iloc[0].tolist()

    # Initialize variables
    datasets = {}
    current_dataset = []
    dataset_counter = 1
    note_counter = 0

    # Process the remaining rows
    for index, row in remaining_df.iterrows():
        if row[0] == "Note":
            note_counter += 1
            if note_counter > 1:
                datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)
                dataset_counter += 1
                current_dataset = []
        current_dataset.append(row)

    # Append the last dataset if exists
    if current_dataset:
        datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)

    # Create a dataframe for each dataset and show the names of the dataframes created
    dataframe_names = []
    for key, dataset in datasets.items():
        globals()[key] = dataset
        dataframe_names.append(key)

    # Apply the operations to each dataset
    for key, dataset in datasets.items():
        # Convert the second column to datetime, coercing errors
        dataset.iloc[3:, 1] = pd.to_datetime(dataset.iloc[3:, 1], format='%d/%m/%Y %H:%M:%S', errors='coerce')
        
        # Replace all commas with periods in columns 2 to the end
        dataset.iloc[:, 1:] = dataset.iloc[:, 1:].replace(',', '.', regex=True)
        
        # Convert subsequent columns to float, coercing errors
        for col in dataset.columns[2:]:
            dataset.iloc[2:, dataset.columns.get_loc(col)] = pd.to_numeric(dataset.iloc[2:, dataset.columns.get_loc(col)], errors='coerce')
        
        # Update the dataset in the dictionary
        datasets[key] = dataset

    # Create the dataframe for storing general data
    riassunto = pd.DataFrame(columns=index_row)
    new_columns = []
    riassunto = pd.concat([riassunto, pd.DataFrame(columns=new_columns)], axis=1)

 # Calculate and assign time delta values for the second column of riassunto
    for key, dataset in datasets.items():
        # Ensure the values are Timestamps
        start_time = pd.to_datetime(dataset.iloc[3, 1], errors='coerce')
        end_time = pd.to_datetime(dataset.iloc[-1, 1], errors='coerce')
        time_delta = end_time - start_time
        row_index = dataframe_names.index(key)
        riassunto.at[row_index, riassunto.columns[1]] = time_delta

    # Assign values from datasets to riassunto sequentially in column 1
    row_counter = 0
    for key, dataset in datasets.items():
        value_to_assign = dataset.iloc[1, 0]  # Value from column 1 row 2
        riassunto.at[row_counter, 'Note'] = value_to_assign
        row_counter += 1

    # Iterate over each dataset
    for key, dataset in datasets.items():
        # Calculate the average values for each column starting from the third column
        averages = dataset.iloc[2:, 2:].mean()
        
        # Assign the average values to the corresponding row in riassunto
        row_index = dataframe_names.index(key)
        for col_index, avg_value in enumerate(averages, start=2):
            riassunto.iloc[row_index, col_index] = avg_value

    # Create new columns for minimum and maximum values
    avg_columns = []
    min_columns = []
    max_columns = []

    for col in riassunto.columns[1:]:
        avg_columns.append(f"{col}_std")
        min_columns.append(f"{col}_min")
        max_columns.append(f"{col}_max")
        

    # Append new columns to riassunto dataframe
    riassunto = pd.concat([riassunto, pd.DataFrame(columns=avg_columns + min_columns + max_columns)], axis=1)

    # Iterate over each dataset
    for key, dataset in datasets.items():
        # Calculate the minimum and maximum values for each column starting from the third column
        deviations = dataset.iloc[2:, 2:].std()
        min_values = dataset.iloc[2:, 2:].min()
        max_values = dataset.iloc[2:, 2:].max()
        
        # Assign the minimum values to the corresponding row in riassunto
        row_index = dataframe_names.index(key)
        for col_index, deviations in enumerate(deviations, start=2):
            riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = deviations
        
        for col_index, min_value in enumerate(min_values, start=2):
            riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = min_value
        
        # Assign the maximum values to the corresponding row in riassunto
        for col_index, max_value in enumerate(max_values, start=2):
            riassunto.iloc[row_index, 2*len(riassunto.columns)//3 + col_index] = max_value


    # Assign 'File name' and 'Last modification' values from row 20 of each dataframe
    riassunto['File name'] = df.at[20,'File name']
    riassunto['Last modification'] = df.at[20,'Last modification']
    #riassunto.at[20, 'File name'] = df.at[20, 'File name']
    #riassunto.at[20, 'Last modification'] = df.at[20, 'Last modification']
    updated_riassunto = append_columns_to_riassunto(riassunto, test_data_columns, test_data_values) 


    # Assign the riassunto dataframe to a variable name sequentially
    globals()[f'riassunto_{i+1}'] = updated_riassunto
    riassunto_dataframes.append(updated_riassunto)

for i in range(1, len(globals())):
    var_name = f'riassunto_{i}'
    if var_name in globals():
        globals()[var_name] = process_dataframe(globals()[var_name])

# Print the processed riassunto dataframes for verification
for i in range(1, len(globals())):
    var_name = f'riassunto_{i}'

header = riassunto_1.columns.tolist()
# Loop through each DataFrame and replace column names with values from 'header'
for i, df in enumerate(riassunto_dataframes):
    df.columns = header

for i in range(len(riassunto_dataframes)):
    riassunto_dataframes[i]=riassunto_dataframes[i].loc[~riassunto_dataframes[i].index.to_series().duplicated(keep='first')]

final_dataframe = pd.concat(riassunto_dataframes, ignore_index=True)

# Sort the dataframe by 'DATA/ORA COLLAUDO'
final_dataframe.sort_values(by='Last modification', inplace=True)

# Create a new column 'numero.test' with sequential values based on the sorted dates
final_dataframe['numero.test'] = final_dataframe.groupby('File name').ngroup().apply(lambda x: f'P{x+1:06d}')

# Ensure that rows with the same 'File Name' have the same 'numero.test'
final_dataframe['numero.test'] = final_dataframe.groupby('File name')['numero.test'].transform('first')

In [ ]:
import tkinter as tk
from tkinter import filedialog
def count_files(code_list):
    code_count = {}
    for code in code_list:
        if code in code_count:
            code_count[code] += 1
        else:
            code_count[code] = 1
    return code_count
code_counts = count_files(skipped_files)
print(code_counts)

def count_items(item_list):
    return len(item_list)
item_count = count_items(skipped_files)

skipped = pd.DataFrame(code_counts, columns=["Code"])

# Open a file dialog to select the directory to save the CSV file
file_path2 = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

# Check if a file path was selected
if file_path2:
    # Export the dataframe as a CSV file
    skipped.to_csv("output.csv", index=False)
    print(f"The list has been successfully transformed into a dataframe and exported as {file_path2}.")
else:
    print("No file path was selected.")





{'00312386_TALA8NMSBA10000R03_20230206_105757.txt': 1, '00312386_TALA8NMSBA10000R03_20230206_115156.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230213_070657.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230213_113726.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230214_085849.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230215_070409.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230216_070625.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230216_110150.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230220_123518.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230224_105251.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230224_113632.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230227_080206.txt': 1, '00312855_PROTOTIPO_TALA3_GROBR00_20230228_080215.txt': 1, '00314070_TAO93NMIBXGT100R01_20230223_145203.txt': 1, '00314070_TAO93NMIBXGT100R01_20230224_070230 - Copia.txt': 1, '00314070_TAO93NMIBXGT100R01_20230224_070230.txt': 1, '00314070_TAO93NMIBXGT100R01_20230225_070746.txt': 1, '00314804_TAL37NBS

44